In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
import plotly.express as px
import os


In [2]:
df = pd.read_excel("Django_Objects.xlsx")
df

,Unnamed: 0,city,calendar_week,raw_entries,clean_entries,avg_price_sqrm,avg_size,avg_year
0,0,Rovinj,30,544,337,4279,91,1900
1,1,Zagreb,30,5836,4122,2616,92,2022
2,2,Varazdin,31,100,54,1735,66,1968
3,3,Varazdin,32,100,56,1740,67,1968
4,4,Zagreb,32,5966,4138,2635,92,2022
5,5,Varazdin,32,100,53,1737,67,1968
6,6,Rijeka,33,1779,1319,1996,92,2020
7,7,Rovinj,33,569,337,4240,94,1900
8,8,Split,33,656,570,3616,85,2000
9,9,Varazdin,33,100,51,1813,77,2023


In [ ]:
fig = px.line(df, x="calendar_week", y="avg_price_sqrm", color = 'city', markers=True, height=600)
fig

In [5]:
fig = px.line(df, x="calendar_week", y="avg_size", color = 'city', markers=True, height=600)
fig


In [3]:
#PLOTLY FOR DJANGO OBJECTS
df = pd.read_excel("Django_Objects.xlsx")

for grad in list(df["city"].unique()):
    fig = px.line(df[df["city"] == grad], x="calendar_week", y="avg_price_sqrm", color = 'city', markers=True, title=grad)
    fig.show()

In [4]:
for grad in list(df["city"].unique()):
    fig = px.line(df[df["city"] == grad], x="calendar_week", y="raw_entries", color = 'city', markers=True, title=grad)
    fig.show()

In [6]:
for grad in list(df["city"].unique()):
    fig = px.line(df[df["city"] == grad], x="calendar_week", y="clean_entries", color = 'city', markers=True, title=grad)
    fig.show()

In [20]:
#PLOTLY FOR DJANGO OBJECTS
df = pd.read_excel("Django_Objects.xlsx")

fig = px.line(df, x="calendar_week", y="avg_price_sqrm", color = 'city', markers=True, title=grad, width=1000, height=600)
fig.show()

In [19]:
import os
cwd = os.getcwd()
onlyfiles = [f for f in listdir(cwd) ]
onlyfiles


['dashboard',
 'db.sqlite3',
 'manage.py',
 'media',
 'raw',
 'RealEstateApp',
 'RealEstateApp.code-workspace',
 'static']

In [4]:
cwd = os.getcwd()

path = cwd + "\\raw"
onlyfiles = [f for f in listdir(path)]

for file in onlyfiles:
    file_path = path + "\\" + file
    print(file_path)

c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-22_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-27_RAW_rovinj.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-27_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-07-27_RAW_zagreb.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-05_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-07_RAW_rovinj.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-07_RAW_varazdin.xlsx
c:\Users\borna.molnar-erhatic\OneDrive - Swietelsky AG\Dokumente\06 Real Estate\RealEstateApp\raw\2022-08-08_RAW_rovi

In [20]:
path = cwd + "\\raw"
onlyfiles = [f for f in listdir(path) ]
onlyfiles

['2022-07-22_RAW_varazdin.xlsx',
 '2022-07-27_RAW_rovinj.xlsx',
 '2022-07-27_RAW_varazdin.xlsx',
 '2022-07-27_RAW_zagreb.xlsx',
 '2022-08-05_RAW_varazdin.xlsx',
 '2022-08-07_RAW_rovinj.xlsx',
 '2022-08-07_RAW_varazdin.xlsx',
 '2022-08-08_RAW_rovinj.xlsx',
 '2022-08-11_RAW_rovinj.xlsx',
 '2022-08-11_RAW_varazdin.xlsx',
 '2022-08-12_RAW_rovinj.xlsx',
 '2022-08-12_RAW_zagreb.xlsx',
 '2022-08-14_RAW_varazdin.xlsx',
 '2022-08-15_RAW_rijeka.xlsx',
 '2022-08-15_RAW_rovinj.xlsx',
 '2022-08-15_RAW_split.xlsx',
 '2022-08-19_RAW_varazdin.xlsx',
 '2022-08-19_RAW_zagreb.xlsx',
 '2022-08-24_RAW_rijeka.xlsx',
 '2022-08-24_RAW_rovinj.xlsx',
 '2022-08-24_RAW_split.xlsx',
 '2022-08-24_RAW_varazdin.xlsx',
 '2022-08-24_RAW_zagreb.xlsx',
 '2022-08-30_RAW_rijeka.xlsx',
 '2022-08-30_RAW_rovinj.xlsx',
 '2022-08-30_RAW_split.xlsx',
 '2022-08-30_RAW_varazdin.xlsx',
 '2022-08-30_RAW_zagreb.xlsx',
 '2022-09-04_RAW_rijeka.xlsx',
 '2022-09-04_RAW_rovinj.xlsx',
 '2022-09-04_RAW_split.xlsx',
 '2022-09-04_RAW_varazdin

In [22]:
file_path = path + "\\" + onlyfiles[0]
df = pd.read_excel(file_path)

In [23]:
df

,Unnamed: 0,Županija,Grad/općina,Naselje,Stambena površina u m2,Broj soba,Zamjena,Tip oglasa,Prodavač,Detaljnije o kući,...,Površina terase u m2,Namještenost,Površina vrta u m2,Ulaz,Grijanje,email,tel ili mobitel,Tip nekretnine za odmor,O zemljištu,Pozicija poslovnog prostora
0,0,Istarska,Rovinj,Rovinj,50,3-3.5 sobni4+\n,ne,ponuda,pravna osoba,Novogradnja\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Istarska,Rovinj,Rovinj,92,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Istarska,Rovinj,Rovinj,96,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Istarska,Rovinj,Rovinj,210,4+\n,ne,ponuda,pravna osoba,Novogradnja\n,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Istarska,Rovinj,Rovinj,32,3-3.5 sobni\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,539,Istarska,Rovinj,Rovinj,97,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
540,540,Istarska,Rovinj,Rovinj,80,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U zgradi
541,541,Istarska,Rovinj,Rovinj,60,4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
542,542,Istarska,Rovinj,Rovinj,91,2-2.5 sobni4+\n,ne,ponuda,pravna osoba,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
name = onlyfiles[1].split("_")

In [38]:

from datetime import datetime
test = datetime.strptime(name[0], '%Y-%m-%d').date()
test, test.isocalendar()[1]

(datetime.date(2022, 7, 27), 30)

In [31]:
ime = "_" + name[2].split(".")[0]
city_name = name[2].split(".")[0].capitalize()


In [32]:
ime, city_name

('_rovinj', 'Rovinj')